In [1]:
import sys
sys.path.append('../src/')

In [3]:
import pandas as pd
import shadowfleet as sf
from pathlib import Path

PATH = Path('../data')

In [71]:
lloyds = pd.read_csv(PATH / 'gfw/shadowfleet_lloyds.csv')

In [74]:
lloyds.dropna(subset=['IMO'], inplace=True)
lloyds.IMO = lloyds.IMO.astype(int)


In [75]:
imos = lloyds.IMO.unique()
len(imos)

667

In [76]:
lloyds_vessels, lloyds_owners = sf.get_vessels(query=imos,
                                filename=PATH / 'gfw/gfw_lloyds.json',
                                field='imo',
                                limit=5)

In [83]:
vessels = lloyds_vessels.copy()
vessels['transmissionDateTo'] = pd.to_datetime(vessels['transmissionDateTo'], errors='coerce')
filtered_vessels = vessels[vessels['transmissionDateTo'] >= '2022-12-01'].copy()
filtered_vessels.dropna(subset='positionsCounter', inplace=True)
filtered_vessels.query('positionsCounter > 1000', inplace=True)
len(filtered_vessels)

1106

In [89]:
filtered_vessels.imo.value_counts()

imo
9257814    4
9233222    4
9281011    4
9371608    4
9297319    4
          ..
9256913    1
9256236    1
9255945    1
9255842    1
9832559    1
Name: count, Length: 605, dtype: int64

In [91]:
filtered_vessels.query('imo=="9257814"')

,id,sourceCode,MMSI,flag,shipname,nShipname,callsign,imo,latestVesselInfo,transmissionDateFrom,transmissionDateTo,geartypes,lengthM,tonnageGt,vesselInfoReference,extraFields,query,messagesCounter,positionsCounter,matchFields
886,578b7053d-d898-1683-bc21-72f6cc1f3a45,[AIS],314981000,BRB,TOREX,TOREX,8P2487,9257814,NaN,2024-10-16T05:47:18Z,2025-01-14 23:57:56+00:00,NaN,NaN,NaN,NaN,NaN,None,271757.0,79817.0,NO_MATCH
887,c925f2f5b-bd74-f700-ce7e-ca582ec01ff2,[AIS],518999188,COK,TOREX,TOREX,E5U5169,9257814,NaN,2024-04-30T09:47:30Z,2024-10-16 05:44:23+00:00,NaN,NaN,NaN,NaN,NaN,None,907313.0,241798.0,NO_MATCH
888,41afed42f-f004-9846-47ce-ecfce71ad02c,[AIS],352002318,PAN,TOREX,TOREX,3E2252,9257814,NaN,2023-01-04T08:34:53Z,2023-07-12 17:49:04+00:00,NaN,NaN,NaN,NaN,NaN,None,424794.0,128471.0,NO_MATCH
889,8015e7e76-6f3d-9d7e-7666-bfa420d69937,[AIS],511101054,PLW,TOREX,TOREX,T8A4287,9257814,NaN,2023-06-21T08:11:25Z,2024-04-30 09:26:30+00:00,NaN,NaN,NaN,NaN,NaN,None,780218.0,276819.0,NO_MATCH


In [84]:
filtered_vessels.ssvid.nunique()

1101

In [85]:
filtered_vessels.rename(columns={'ssvid': 'MMSI'}, inplace=True)

In [93]:
filtered_vessels[500:].to_csv(PATH / 'gfw/shadowfleet_lloyds_for_gfw_second_half.csv', index=False)